In [2]:
import os
import collections
import pandas as pd
import xmltodict
from IPython.display import display
from tqdm import tqdm
from collections import OrderedDict

def runProcess():
    dict_results={}
    dict_results['Medical Specialty'] = []
    dict_results['Disorders'] = []
    list_of_files = []
    working_path = '../xml/'
    os.chdir(working_path)
    for xmlfile in os.listdir(os.getcwd()):
        if xmlfile.find('_product3_') != -1:
            list_of_files.append(xmlfile)
    MedicalSpecialties = {}
    for xmlfile in tqdm(list_of_files):
        full_classif_list = []
        path_in = working_path + '/' + xmlfile
        xml_dict = getData(path_in)
        full_classif_list.extend(explore_classif(xml_dict))
        ListDisorderTypeId = ['21429', '21401', '21415', '21394', '21422', '21408']
        MedicalSpecialtyOrphacodes = []
        for classif in full_classif_list:
            if classif['DisorderTypeId'] in ListDisorderTypeId:
                MedicalSpecialtyOrphacodes.append(classif["ORPHAcode"])
        MedicalSpecialtyOrphacodes = set(MedicalSpecialtyOrphacodes)
        MedicalSpecialties[full_classif_list[0]['Label']] = len(MedicalSpecialtyOrphacodes)
    MedicalSpecialties = OrderedDict(sorted(MedicalSpecialties.items()))
    for medicalSpecialty in MedicalSpecialties:
        dict_results['Medical Specialty'].append(medicalSpecialty)
        dict_results['Disorders'].append(MedicalSpecialties[medicalSpecialty])
    myDataframe = pd.DataFrame(dict_results)
    with pd.ExcelWriter('../output_tables/MedicalSpecialties.xlsx', engine='xlsxwriter') as writer:
        myDataframe.to_excel(writer,index=False)  
    display(myDataframe.head(len(dict_results['Medical Specialty'])))


def getData(xmlfile):
    """
    Read an xml return a dict with xmltodict package
    :return: xml parsed as dict
    """
    with open(xmlfile, "r", encoding='ISO-8859-1') as ini:
        xml_dict = xmltodict.parse(ini.read())
    return xml_dict



class ClassifLineDict(collections.OrderedDict):
    """
    Instantiate an ordered dict that will be used to keep the order of the column in the EXCEL file for classif sheet
    """

    def __init__(self):
        super(ClassifLineDict, self).__init__()
        self["ORPHAcode"] = 0
        self["OrphId"] = 0
        self["Label"] = ""
        self["DisorderType"] = ""
        self["DisorderTypeId"] = ""


def recursive_explore(tree, depth):
    """
    Will explore the classification to fill the global list_classif_out with ClassifLineDict() for each disorder
    :param tree: current disorder
    :param depth: int, depth in the classification tree
    :return: None
    """
    if isinstance(tree, dict):
        leaf = ClassifLineDict()
        leaf["ORPHAcode"] = int(tree["Disorder"]["OrphaCode"])
        leaf["OrphId"] = int(tree["Disorder"]["@id"])
        leaf["Label"] = tree["Disorder"]["Name"]["#text"]
        if "DisorderType" in tree["Disorder"].keys():
            leaf["DisorderType"] = tree["Disorder"]["DisorderType"]["Name"]["#text"]
            leaf["DisorderTypeId"] = tree["Disorder"]["DisorderType"]["@id"]
        list_classif_out.append(leaf)
        if tree["ClassificationNodeChildList"]["@count"] != "0":
            if  isinstance(tree["ClassificationNodeChildList"]["ClassificationNode"], list):
                for node in tree["ClassificationNodeChildList"]["ClassificationNode"]:
                    if "@" in node:
                        pass
                    recursive_explore(node, depth + 1)
            else:
                recursive_explore(tree["ClassificationNodeChildList"]["ClassificationNode"], depth + 1)

def explore_classif(xml_dict):
    """
    Will explore the classification xml to fill ClassifLineDict() for each disorder
    :param xml_dict:
    :return: list_classif_out = [ClassifLineDict(), ...]
    """
    root = xml_dict["JDBOR"]["ClassificationList"]["Classification"]
    classif_head = ClassifLineDict()
    classif_head["ORPHAcode"] = int(root["OrphaNumber"])
    classif_head["OrphId"] = int(root["@id"])
    classif_head["Label"] = root["Name"]["#text"]
    global list_classif_out
    list_classif_out = [classif_head, ]
    tree = root["ClassificationNodeRootList"]["ClassificationNode"]
    recursive_explore(tree, depth=0)
    return list_classif_out


if __name__ == '__main__':

    runProcess()

100%|██████████| 33/33 [00:03<00:00, 10.45it/s]


,Medical Specialty,Disorders
0,Orphanet classification of rare abdominal surg...,160
1,Orphanet classification of rare allergic disease,14
2,Orphanet classification of rare bone diseases,804
3,Orphanet classification of rare cardiac diseases,149
4,Orphanet classification of rare cardiac malfor...,172
5,Orphanet classification of rare circulatory sy...,209
6,Orphanet classification of rare developmental ...,2696
7,Orphanet classification of rare disorders due ...,55
8,Orphanet classification of rare endocrine dise...,446
9,Orphanet classification of rare gastroenterolo...,200
